In [4]:
import os
import json
import pandas as pd
import numpy as np
from feature_extraction import *

In [5]:
data_dir = '../../data'
pxm = 'p6m'

df_reg = pd.read_csv(os.path.join(data_dir, 'model/{}/full.csv'.format(pxm)))
val = pd.read_csv(os.path.join(data_dir, 'model/{}/validate.csv'.format(pxm)))
test = pd.read_csv(os.path.join(data_dir, 'model/{}/test.csv'.format(pxm)))
test = test.append(val)

dir_feat = 'data/predictions'
feats = []
for fname in os.listdir(dir_feat):
    tmp = pd.read_csv(os.path.join(dir_feat, fname))
    tmp['model'] = fname.split('.')[0]
    feats.append(tmp[['p1', 'id', 'model']])

In [6]:
cids = {}

fname = os.path.join(data_dir, 'main/currentacct_txn.csv')
df_trans = read_current(fname, df_reg['id'])
cids['current'] = df_trans['id'].unique()

fname = os.path.join(data_dir, 'main/creditcard_txn.csv')
df_trans = read_credit(fname, df_reg['id'])
cids['credit'] = df_trans['id'].unique()

fname = os.path.join(data_dir, 'main/savingacct_txn.csv')
df_trans = read_saving(fname, df_reg['id'])
cids['saving'] = df_trans['id'].unique()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [28]:
df_preds = pd.concat(feats).pivot(index='id', columns='model', values='p1')

for acc_type in cids:
    for col in [acc_type + '_diff', acc_type + '_transact']:
        df_preds.loc[~df_preds.index.isin(cids[acc_type]), col] = np.nan
        
df_preds = df_preds[~(df_preds.isna().sum(axis=1) == 6)]

counts = (~pd.isna(df_preds)).sum(axis=1)
df_preds['ensemble'] = df_preds.sum(axis=1) / counts
df_preds = df_preds \
    .reset_index(drop=False) \
    .melt(id_vars='id', value_vars=df_preds.columns, var_name='model', value_name='value')

df_preds = df_preds[~df_preds['value'].isna()]

tmp = df_preds['model'].str.split('_')
df_preds['model'] = tmp.str[0]
df_preds['feature'] = tmp.str[1]

tmp = df_preds[df_preds['model'] == 'ensemble']
final_pred_lookup = dict(zip(tmp['id'], tmp['value']))

df_preds = df_preds[df_preds['model'] != 'ensemble']

df_preds['feature'] = df_preds['feature'].replace({'diff': 'Consistency', 'transact': 'Amount'})
df_preds['model'] = df_preds['model'].replace({'credit': 'Credit Card', 'saving': 'Saving Acc', 'current': 'Current Acc'})

df_preds = df_preds.rename({'feature': 'Feature', 'model': 'Account', 'value': 'Value'}, axis=1)

In [29]:
max(list(final_pred_lookup.keys()), key=lambda x: final_pred_lookup[x])

20858441

In [48]:
final_pred_lookup[cid]

0.25619824623143184

In [44]:
cid = 20858441
df = df_preds[df_preds['id'] == cid]
fig = px.bar(
    df, x="Account", y="Value", 
    color="Feature", barmode="group",
    color_discrete_sequence=['#0664ac', '#6dc7dc'],
    title="Voting"
)
fig.show()

In [47]:
path = os.path.join(data_dir, 'demo', 'algo', 'votes.csv')
df_preds.to_csv(path, index=False)

In [49]:
path = os.path.join(data_dir, 'demo', 'algo', 'votes_combined.json')
with open(path, 'w') as f:
    json.dump(final_pred_lookup, f)